In [ ]:
#include <iostream>
#include <thread>
#include <vector>
#include <iostream>
#include <atomic>
#include <set>
#include <string>

#include "compapi.hpp"
#include "infoapi.hpp"

const int N = 100;
// S(N) = 1 + 2*2 + 3*3 + 4*4 + .. + (N-1)*(N-1) + N*N
{
    int S = 0;
    for(int n = 1; n <= N; n++) S = S + n*n;
    cout << "S(" << N << ") = " << S << endl;
}

In [ ]:
// 1 //
{
    atomic_int PID = 0;
    mutex mut;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        int pid = PID++;
    //////////////// inside a process ////////////////
        struct SumSquareWorker: public ComputeWorker{
            SumSquareWorker(unsigned pid, EventLog&log, int&_S):ComputeWorker(pid,log),S(_S){
                for(int n = 1; n <= N; n++) not_processed.insert(n);
                S = 0;
            }
            bool on_read(unsigned ord,unsigned tag,unsigned pid,const BLOB&blob)override{
                if(processed.find(tag)==processed.end()){
                    int n_by_n = stoi(blob);
                    S = S + n_by_n;
                    processed.insert(tag);
                    not_processed.erase(tag);
                }
                return (processed.size()!=N);
            }
            bool on_write(unsigned&tag,BLOB&blob)override{
                int size = not_processed.size();
                if(size==0) return false;

                int selected = rand()%size;
                set<int>::iterator it = not_processed.begin();
                for(int i=0; i!=selected; i++,it++);
        
                int n = *it; 
                tag = n; blob = to_string(n*n); 
                not_processed.erase(it);
                return true;
            }
            int& S;
            set<int> processed;
            set<int> not_processed;
        };

        int S = 0;
        SumSquareWorker worker(pid,log,S);
        worker.run();
        {
            unique_lock lk(mut);
            cout << "pid = " << pid << " S(" << N << ") = " << S << endl;
        }    
    //////////////////////////////////////////////////
    }); 
    for(auto& t:threads) t.join();
}

In [ ]:
// 2 //
{
    atomic_int PID = 0;
    mutex mut;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        int pid = PID++;
    //////////////// inside a process ////////////////
        struct SumSquareTask: public Task{
            SumSquareTask(TaskEngine&te, int _n):Task(te),n(_n){}
            void on_run()override{ n = n*n; }
            void on_save(ostream&out)override{ out << n; }
            void on_load(istream&in)override{ in >> n; }
            int  n;
        };
        
        TaskEngine eng(pid,log);
        
        vector<SumSquareTask*> tasks(N);
        
        for(int n=1;n<=N;n++){
            tasks[n-1] = new SumSquareTask(eng,n);
            tasks[n-1]->submit();
        }

        eng.run();
        
        {
            unique_lock lk(mut);
            int S = 0;
            for(auto& t:tasks){ S = S + t->n;  delete t; }
            cout << "pid = " << pid << " S(" << N << ") = " << S << endl;
        }
    //////////////////////////////////////////////////
    });
    for(auto& t:threads) t.join();
}

In [ ]:
// 3
{
    atomic_int PID = 0;
    mutex mut;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        int pid = PID++;
    //////////////// inside a process ////////////////   
        struct SquareParam{ int  n; };
        
        struct SumSquarePS: public ParamSweep<SquareParam>{
            SumSquarePS(unsigned pid,EventLog& log):
                ParamSweep(pid,log),n(1),S(0){}
        
            bool on_get(SquareParam&t)override{ if(n>N)return false; t.n = n; n++; return true; }
            void on_run(SquareParam&t)override{ t.n = t.n * t.n; }
            void on_put(SquareParam&t)override{ S = S + t.n; }
            void on_save(const SquareParam&t,ostream&out)override{ out << t.n; }
            void on_load(SquareParam&t,istream&in)override{ in >> t.n; }
            int n; int S;
        };
        
        SumSquarePS param_sweep(pid,log);
        param_sweep.run();
        {
            unique_lock lk(mut);
            cout << "pid = " << pid << " S(" << N << ") = " << param_sweep.S << endl;
        }
    //////////////////////////////////////////////////
    });
    for(auto& t:threads) t.join();
}

In [ ]:
// 4
{
    atomic_int PID = 0;
    mutex mut;
    int num_threads = 10;
    
    EventLog log;
    vector<thread> threads(num_threads);
    
    for(auto& t:threads)t=thread([&]{
        int pid = PID++;
    //////////////// inside a process ////////////////      
        class SumSquareBag: public Bag<int>{
        public:
            SumSquareBag(unsigned pid, EventLog& log):Bag(pid,log),S(0),cur_n(1){}
        protected:
            bool on_get(int&n)override{ if(cur_n <= N) { n = cur_n++; return true;} else return false; }
            void on_put(int&n)override{ S = S + n; }
            void on_save(const int&n,ostream&out)override{ out << n; }
            void on_load(int&n,istream&in)override{ in >> n; }
        private:
            int cur_n;
        public:
            int S;
        } bag(pid,log);

        int n;
        while(bag.get(n)){
            n = n*n;
            bag.put(n);
        }
        {
            unique_lock lk(mut);
            cout << "pid = " << pid << " S(" << N << ") = " << bag.S << endl;
        }
    //////////////////////////////////////////////////
    });
    for(auto& t:threads) t.join();
}